## Trabalho – Mineração de Dados – DSM - Prof. Adriana Talhari

### Para usar este ``notebook``faça:
1. Crie o ambiente '.venv'
``python3 -m venv .venv``
2. Ative o ambiente (se não ativar sozinho)
``source .venv/bin/activate``
3. Instale as dependências
``pip install -r requirements.txt``

### Para as questões de 1 @ 4.

In [ ]:
import pandas as pd
import numpy as np

# 1. Carregar o dataset definindo '?' como NaN (Not a Number)
df = pd.read_csv("wine.csv", na_values="?")

# --- Respostas para as Questões 1 e 2 ---
qtd_atributos = df.shape[1]  # Colunas
qtd_instancias = df.shape[0]  # Linhas

print(f"1) Quantos Atributos existem no dataset? R: {qtd_atributos}")
print(f"2) Quantas Instâncias existem no dataset? R: {qtd_instancias}")

# --- Resposta para a Questão 3 ---
# Identificar colunas com valores nulos
atributos_ausentes = df.columns[df.isnull().any()].tolist()
print(f"\n3) Quais os dois atributos com dado ausente: R: {atributos_ausentes}")

# --- Resposta para a Questão 4 ---
# Calcular a média desses atributos (ignorando os NaNs automaticamente)
print("\n4) Qual a média dos valores dos dois atributos identificados?")
for attr in atributos_ausentes:
    media = df[attr].mean()
    print(f"Atributo: {attr} - Média: {media:.4f}")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 1. Preencher valores ausentes com a média (ReplaceMissingValues)
df_filled = df.fillna(df.mean())

# 2. Separar as Features (X) da Classe (y)
# A classe é a última coluna "Class_Label"
X = df_filled.drop("Class_Label", axis=1)
y = df_filled["Class_Label"].astype(int)  # Convertendo para Nominal/Inteiro

# 3. Normalizar todos os valores entre 0 e 1 (Normalize)
scaler = MinMaxScaler()
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

print("Pré-processamento concluído: Dados preenchidos e normalizados.")

#### Questão 5

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Definindo os 3 métodos de ligação solicitados
methods = [
    ('Complete', 'complete'), # Distância Euclidiana é padrão
    ('Average', 'average'),
    ('Ward', 'ward')
]

print("\n--- 5) Agrupamento Hierárquico (3 Clusters) ---")

best_method = ""
best_score = 0

for name, linkage_type in methods:
    # Criar e treinar o modelo
    model = AgglomerativeClustering(n_clusters=3, linkage=linkage_type, metric='euclidean')
    labels = model.fit_predict(X_normalized)
    
    # Avaliação: Cruzar os Clusters Encontrados vs Classes Reais
    # Isso simula o "Classes to clusters evaluation"
    contingency_table = pd.crosstab(y, labels, rownames=['Real Class'], colnames=['Cluster'])
    
    print(f"\n>> Método de Ligação: {name}")
    print(contingency_table)
    print("-" * 30)

# Observação: Analise visualmente qual tabela agrupa melhor as classes originais (0, 1, 2)
# em clusters distintos (0, 1, 2). O método WARD geralmente tende a funcionar melhor em datasets estruturados.

#### Questão 6

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Configuração da Validação Cruzada (10 Folds)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Definindo os modelos
modelos = [
    ('Árvore de Decisão (J48)', DecisionTreeClassifier(criterion='entropy', random_state=42)),
    ('LDA', LinearDiscriminantAnalysis()),
    ('Naive Bayes', GaussianNB())
]

print("\n--- 6) Classificação (Cross-Validation 10 Folds) ---")

resultados = {}

for nome, modelo in modelos:
    # Executar o cross-validation e pegar a acurácia
    scores = cross_val_score(modelo, X_normalized, y, cv=cv, scoring='accuracy')
    media_acc = scores.mean() * 100
    resultados[nome] = media_acc
    
    print(f"\nModelo: {nome}")
    print(f"Acurácia Média: {media_acc:.2f}%")
    print(f"Desvio Padrão: {scores.std():.4f}")

# Identificar o melhor resultado
melhor_modelo = max(resultados, key=resultados.get)
print(f"\n>> O modelo que apresentou o melhor resultado foi: {melhor_modelo} com {resultados[melhor_modelo]:.2f}%")